In [1]:
import sys
import serial
import time
import cv2
import our_library as lib
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import Rbf

from PyQt5.QtCore import pyqtSlot
from PyQt5.QtWidgets import QApplication, QDialog
from PyQt5.uic import loadUi
from PyQt5.QtWidgets import QApplication
from PyQt5.QtWidgets import QWidget
from PyQt5.QtGui import QImage
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QTimer
from PyQt5.QtCore import Qt



###################### Zaznane koordinate objekta pretvori v koordinate DMX s pomočjo interpolacije ##############

def Kalib_DMX(iKoorX,iKoorY, luc_na_sliki, DMX):
    # Razmerje iz slike kamere, v koordinatni sistem luči
    # luc_na_sliki je matrika težišč luci zaznanih na kameri
    x = luc_na_sliki[:,0]
    y = luc_na_sliki[:,1]
    pan_z = DMX[:,0]
    tilt_z = DMX[:,1]

    # use RBF
    rbf1 = Rbf(x, y, pan_z, epsilon=2)
    phi = rbf1(iKoorX, iKoorY) # cX cY (tocke iz sledenja)
    rbf2 = Rbf(x, y, tilt_z, epsilon=2)
    theta = rbf2(iKoorX, iKoorY)
    
    return int(phi),int(theta)




###################### Funkcija za formatiranje stringa in posiljanje na Arduino #######################

def formatiranje(dimmer, pan, tilt, color):
    
    # pan -> 0 - 65535
    
    # Koordinati x in y za vodenje luci
    iPan = pan//256
    iPanFine = int(((pan/256) - iPan)*256)
    iTilt = tilt//256
    iTiltFine = int(((tilt/256) - iTilt)*256)
    
    # dimmer -> svetlost luci
    # color -> izbira barve luci
    iDimmer = str(dimmer)
    iColor = str(color)
    iPan = str(iPan)
    iPanFine = str(iPanFine)
    iTilt = str(iTilt)
    iTiltFine = str(iTiltFine)

    # Ce dolzina stringa ni enaka 3 znaki, dodaj spredaj niclo, da bo
    while (len(iDimmer) < 3):
        iDimmer = '0' + iDimmer
    while (len(iPan) < 3):
        iPan = '0' + iPan
    while (len(iPanFine) < 3):
        iPanFine = '0' + iPanFine
    while (len(iTilt) < 3):
        iTilt = '0' + iTilt
    while (len(iTiltFine) < 3):
        iTiltFine = '0' + iTiltFine
    while (len(iColor) < 3):
        iColor = '0' + iColor

    # Celoten string, ki ga arduino zna desifrirati in posredovati na DMX (luc)
    celota =  iDimmer + iPan + iPanFine + iTilt + iTiltFine + iColor + '!'
    return celota

In [2]:
# zacetni pogoji
counter = 0
counter1 = 0
# izbira točk v vrstici za generiranje matrike točk, po katerih se bo premikala luč za kalibracijo
tocke = 10
# pripravimo array dolzine tock v vrstici
luc_na_sliki = lib.ogljisca_slika(tocke)
# nalozimo haarcascade classifier za zaznavanje obraza
faceCascade = cv2.CascadeClassifier(r'C:\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')


###################### Zacetek GUI-ja ######################
class MainPage(QDialog):
    def __init__(self):
        super(MainPage, self).__init__()
        # Zazene GUI, ki je bil narejen v QT designer
        loadUi('Test_spremenjen.ui', self)
        # Gumbi pozicije
        self.Poz_1.clicked.connect(self.retriveValue_Poz1)
        self.Poz_2.clicked.connect(self.retriveValue_Poz2)
        self.Poz_3.clicked.connect(self.retriveValue_Poz3)
        self.Poz_4.clicked.connect(self.retriveValue_Poz4)
        
        # Apply & close gumbi
        self.buttonBox123.clicked.connect(self.Close)
        self.buttonBox123.clicked.connect(self.Close_serial)
        self.buttonBox123.clicked.connect(self.controlTimer)
        
        # Drsniki
        self.horizontalSlider_pan.valueChanged.connect(self.slajder_pan)
        self.horizontalSlider_tilt.valueChanged.connect(self.slajder_tilt)
        self.verticalSlider.valueChanged.connect(self.slajder_dimmer)
        self.color_slider.valueChanged.connect(self.slajder_color)
        self.horizontalSlider_thresh.valueChanged.connect(self.slajder_threshold)
        self.verticalSlider.valueChanged.connect(self.Poslji_ardu)
        self.horizontalSlider_pan.valueChanged.connect(self.Poslji_ardu)
        self.horizontalSlider_tilt.valueChanged.connect(self.Poslji_ardu)
        self.color_slider.valueChanged.connect(self.Poslji_ardu)
        self.slajder_track.valueChanged.connect(self.slajder_tracking)

        # create a timer
        self.timer = QTimer()
        self.timer.timeout.connect(self.viewCam)
        # set control_bt callback clicked  function
        self.control_bt.clicked.connect(self.controlTimer)
         
        # checkbox
        self.checkBox.stateChanged.connect(self.Ploter)  

    # Funkcija za izris mreže med označenimi ogljišči DMX
    def Ploter(self):
        DMXout = lib.ogljisca_slika(tocke)
        if self.checkBox.isChecked() == True:
            DMX = lib.Meshgrid_DMX(self.Pan_2,
                                   self.Pan_1,
                                   self.Pan_4,
                                   self.Pan_3,
                                   self.Tilt_2,
                                   self.Tilt_1,
                                   self.Tilt_4,
                                   self.Tilt_3,
                                   tocke)
            return DMX
        
    # Funkcija za izris mreže med označenimi ogljišči SLIKA
    def Ploter_slika(self):
        if self.checkBox.isChecked() == True:
            slika_oder = lib.Meshgrid_slika(self.ogljisce_slika1[0],
                                            self.ogljisce_slika2[0],
                                            self.ogljisce_slika3[0],
                                            self.ogljisce_slika4[0],
                                            self.ogljisce_slika1[1],
                                            self.ogljisce_slika2[1],
                                            self.ogljisce_slika3[1],
                                            self.ogljisce_slika4[1],
                                            tocke)
            return slika_oder
    
#     # Izbiranje algoritmov za sledenje (boljši v našem primeru 'BOOSTING','KCF' in 'CSRT')
#     def slajder_tracking(self):
#         word = self.slajder_track.value()
#         word1 = str(word)
#         tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
#         self.label_17.setText(tracker_types[word])
#         return word
    
    def slajder_pan(self):
        word = self.horizontalSlider_pan.value()
        word1 = str(word)
        pan_ardu = word
        self.label_3.setText(word1)
        return pan_ardu
        
    def slajder_tilt(self):
        word = self.horizontalSlider_tilt.value()
        word1 = str(word)
        tilt_ardu = word
        self.label_4.setText(word1)
        return tilt_ardu
    
    def slajder_color(self):
        word = self.color_slider.value()
        word1 = str(word)
        color = word
        self.label_14.setText(word1)
        return color
    
    def slajder_dimmer(self):
        word = self.verticalSlider.value()
        word1 = str(word)
        dimmer = word
        self.label_10.setText(word1)
        return dimmer
    
    def slajder_threshold(self):
        word = self.horizontalSlider_thresh.value()
        word1 = str(word)
        self.label_12.setText(word1)
        
    # Zabeleži trenutno ogljišče
    def retriveValue_Poz1(self):
        self.Pan_1 = self.horizontalSlider_pan.value()
        self.Tilt_1 = self.horizontalSlider_tilt.value()
        self.ogljisce_slika1 = (self.cX, self.cY)
        Pan1 = str(self.Pan_1)
        Tilt1 = str(self.Tilt_1)
        Pan_Tilt1 = Pan1 + ', ' +Tilt1
        self.label_5.setText(Pan_Tilt1)

    def retriveValue_Poz2(self):
        self.Pan_2 = self.horizontalSlider_pan.value()
        self.Tilt_2 = self.horizontalSlider_tilt.value()
        self.ogljisce_slika2 = (self.cX, self.cY)
        Pan2 = str(self.Pan_2)
        Tilt2 = str(self.Tilt_2)
        Pan_Tilt2 = Pan2 + ', ' +Tilt2
        self.label_6.setText(Pan_Tilt2)
      
    def retriveValue_Poz3(self):
        self.Pan_3 = self.horizontalSlider_pan.value()
        self.Tilt_3 = self.horizontalSlider_tilt.value()
        self.ogljisce_slika3 = (self.cX, self.cY)
        Pan3 = str(self.Pan_3)
        Tilt3 = str(self.Tilt_3)
        Pan_Tilt3 = Pan3 + ', ' +Tilt3
        self.label_7.setText(Pan_Tilt3)
        
    def retriveValue_Poz4(self):
        self.Pan_4 = self.horizontalSlider_pan.value()
        self.Tilt_4 = self.horizontalSlider_tilt.value()
        self.ogljisce_slika4 = (self.cX, self.cY)
        Pan4 = str(self.Pan_4)
        Tilt4 = str(self.Tilt_4)
        Pan_Tilt4 = Pan4 + ', ' +Tilt4
        self.label_8.setText(Pan_Tilt4)
        
    def Close(self):
        self.close()
    
    def viewCam(self):
        # read image in BGR format
        _, self.image1 = self.cap.read()
        # convert image to RGB format
        self.image = cv2.cvtColor(self.image1, cv2.COLOR_BGR2RGB)
        
        gray_image = cv2.cvtColor(self.image1, cv2.COLOR_RGB2GRAY)
        gauss = cv2.GaussianBlur(gray_image, (5,5),5)
        # convert the grayscale image to binary image
        threshold = self.horizontalSlider_thresh.value()
        _,thresh = cv2.threshold(gauss,threshold,255,0)
        # find contours in the binary image
        _, contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        # Iskanje tezisca objektov na sliki - prikaži le tistega z najvecjo povrsino (luc)
        max_area = 0
        max_contour_index = 0
        M = []
        for i, contour in enumerate(contours):
            M.append(cv2.moments(contour))
            contour_area = cv2.moments(contour)['m00']
            if contour_area > max_area:
                max_area = contour_area
                max_contour_index = i
                
        cX, cY = 0, 0
        for i in range(len(M)):
            if M[i]["m00"] == max_area:
                cX = int(M[i]["m10"] / M[i]["m00"])
                cY = int(M[i]["m01"] / M[i]["m00"])
            cv2.circle(self.image, (cX, cY), 5, (0, 0, 255), -1)
            cv2.putText(self.image, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # koordinate težišča kroga oz. luči na sliki
        self.cX = cX
        self.cY = cY
            
        if self.checkBox.isChecked() == True:    
            for i in self.Ploter_slika():
                cv2.circle(self.image, tuple(i), 2, (0,255, 0), -1)
                #cv2.putText(self.image, str(i), (tuple(i)[0] - 10, tuple(i)[0]- 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
        # get image infos        
        height, width, channel = self.image.shape
        step = channel * width
        height1, width1 = thresh.shape
        step1 = 1 * width1
        # create QImage from image
        qImg = QImage(self.image.data, width, height, step, QImage.Format_RGB888)
        qImg_thr = QImage(thresh.data, width,height,step/3, QImage.Format_Indexed8)
        # show image in img_label
        self.image_label.setPixmap(QPixmap.fromImage(qImg))
        self.image_threshold.setPixmap(QPixmap.fromImage(qImg_thr))
        
        global counter
        global counter1
        global luc_na_sliki
        
        # izvede kalibracijo. Luč se začne pomikati po zgeneriranih točkah in hkrati zapisuje točke težišča luči
        if self.checkBox_2.isChecked() == True:
                
            if (counter < (len(self.Ploter()) - 1) and counter1 > 0): 
                time.sleep(0.5)
                poslji = formatiranje(255, self.Ploter()[counter+1][0], self.Ploter()[counter+1][1], 0)
                poslji_calib = poslji.encode("utf-8")
                ser.write(poslji_calib)
                
            if (counter < len(self.Ploter()) and counter1 > 0):
                time.sleep(1)
                luc_na_sliki[counter] = self.cX,self.cY
                counter = counter + 1
                
            if counter1 == 0:
                poslji = formatiranje(255, self.Ploter()[counter1][0], self.Ploter()[counter1][1], 0)
                poslji_calib = poslji.encode("utf-8")
                ser.write(poslji_calib)
                counter1 = counter1 + 1
                time.sleep(1)
        
#         if self.checkBox_3.isChecked() == True:

#             print('luc_na_sliki')
#             print(luc_na_sliki)
                
        # Zažene algoritem za sledenje        
        if self.checkBox_4.isChecked() == True:
            
            video = self.cap
            video.set(cv2.CAP_PROP_EXPOSURE,-4) # Disable auto exposure - nastimas svoje http://www.principiaprogramatica.com/2017/06/11/setting-manual-exposure-in-opencv/
            width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
            hight = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

#            
            while (self.checkBox_4.isChecked() == True):

                # Read a new frame
                ok, frame = video.read()
                if not ok:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Start timer
                timer = cv2.getTickCount()

                # Zaženemo algoritem za detekcijo obraza na sliki
                faces = faceCascade.detectMultiScale(
                    frame,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    #flags=cv2.CV_HAAR_SCALE_IMAGE
                )
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # racunanje sredisca kvadrata
                iKoorX = abs(int(x+w//2))
                iKoorY = abs(int(y+h//2))

                # Poslji na arduino kalibrirane koordinate
                pan, tilt = Kalib_DMX(iKoorX,iKoorY, luc_na_sliki, self.Ploter())
                dimmer = 20
                color = 0
                poslji_novo = formatiranje(dimmer, pan, tilt, 0)
                poslji_ready_novo = poslji_novo.encode("utf-8")
                ser.write(poslji_ready_novo)

                # Draw bounding box
                if ok:
                    # Tracking success
                    #p1 = (int(bbox[0]), int(bbox[1]))
                    #p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                    cv2.circle( frame,( iKoorX, iKoorY), 5, (255,0,0), -1 )
                else :
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
                    poslji_novo = formatiranje(11, 24316, 255,0)
                    poslji_ready_novo = poslji_novo.encode("utf-8")
                    ser.write(poslji_ready_novo)

                    cv2.circle( frame,( iKoorX,iKoorY), 5, (255,0,0), -1 ) #BRISI

                height, width = frame.shape
                step = 3 * width
                # create QImage from image
                qImg = QImage(frame.data, width,height,step/3, QImage.Format_Indexed8)
                # show image in img_label
                self.image_label.setPixmap(QPixmap.fromImage(qImg))
                

                # Exit if ESC pressed
                k = cv2.waitKey(1) & 0xff
                if k == 27:
                    cv2.destroyWindow('ROI selector')
            
            
        

    # start/stop timer
    def controlTimer(self):
        # if timer is stopped
        if not self.timer.isActive():
            # create video capture
            self.cap = cv2.VideoCapture(0) # spremeni za webcam ali USB cam
            self.cap.set(cv2.CAP_PROP_EXPOSURE,-4)
            # start timer
            self.timer.start(20)
            self.control_bt.setText("Stop")

        # if timer is started
        else:
            # stop timer
            self.timer.stop()
            # release video capture
            self.cap.release()
            # update control_bt text
            self.control_bt.setText("Start")
            
    def Poslji_ardu(self):
        pan = self.slajder_pan()
        tilt = self.slajder_tilt()
        dimmer = self.slajder_dimmer()
        color = self.slajder_color()
        poslji = formatiranje(dimmer, pan, tilt, color)
        poslji_ready = poslji.encode("utf-8")
        ser.write(poslji_ready)
    
    def Close_serial(self):
        ser.close()
        
    

In [3]:
# Vzpostavi povezavo z Arduinom
ser = serial.Serial('COM7', 115200)

# Zaženemo prej definiran GUI
if not QApplication.instance():
        app = QApplication(sys.argv)
else:
        app = QApplication.instance()
        
widget = MainPage()
widget.show()
MainPage().Poslji_ardu()
app.exec()


0